### Udacity Data Science Nanodegree

### Project 3: Disaster Recovery Pipeline
---

# Notebook 2 of 3: # ETL Pipeline Preparation to translate text to english where necessary

## by Juanita Smith

---

### Note: This notebook is optional (not part of Udacity requirements)


#### Table of contents:
- [1. Introduction](#step1)
- [2. Import libraries and load datasets](#step2)
- [3. Set API key and connect to OpenAI](#step3)
- [4. Build the OpenAI API request in **REALTIME**](#step4)
- [5. Create OpenAI API requests in **BATCHES**](#step5)
- [6. Load and Analyze API batch results into JSON file](#step6)
- [7. Load API results from JSON into pandas dataframe](#step7)
- [8. Clean translations and save to SQLite database](#step8)
- [9. Update messages dataset with translations and update SQLite database](#step9)


<a class="anchor" id=step1></a> 
<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">1. Introduction</span>

During ETL pipeline preparation, it was discovered that some messages were not in English.
Experiment with OpenAI and CHATGPT to detect the language of each message and translate it to English if necessary.

Note: CHATGPT is a paid service.
Just for general experimentation, only a sample of the data will be converted
Completed translations will be stored in json format in `../data/translations/batch_job_results_results.jsonl` and also written to SQLITE database as `message_language`

Finally, messages database will be updated with improved translated messages.

<a class="anchor" id=step2></a> 
<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">2. Import libraries and load datasets</span>

Cleaned `messages` and `message_language` will be loaded from SQLite database.
Dataset `message_language` will keep track of the indexes of messages that were already converted.
When a new translation is requested, messages not yet processed will be selected.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from openai import OpenAI
import json
import numpy as np
import time
from src import config

import logging
logger = logging.getLogger(__name__)

# environment settings
pd.set_option('display.max_column', 400)
pd.set_option('display.max_colwidth', 400)

In [2]:
# activate logging
logging.basicConfig(filename=config.path_log_translation,
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    filemode='w',
                    level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S')

In [3]:
# load data from database created during ETL pipeline preparation
engine = create_engine(config.path_database)
conn = engine.connect()
df = pd.read_sql('select * from messages', con=conn, index_col='id')
df.head()

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that could pass over Haiti,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country today and tonight",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26179 entries, 2 to 30265
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   message                 26179 non-null  object
 1   genre                   26179 non-null  object
 2   related                 26179 non-null  int64 
 3   request                 26179 non-null  int64 
 4   offer                   26179 non-null  int64 
 5   aid_related             26179 non-null  int64 
 6   medical_help            26179 non-null  int64 
 7   medical_products        26179 non-null  int64 
 8   search_and_rescue       26179 non-null  int64 
 9   security                26179 non-null  int64 
 10  military                26179 non-null  int64 
 11  water                   26179 non-null  int64 
 12  food                    26179 non-null  int64 
 13  shelter                 26179 non-null  int64 
 14  clothing                26179 non-null  int64 
 15  money  

In [5]:
# try to read message where language detection was already executed
df_language = pd.DataFrame()
try:
    df_language = pd.read_sql('select * from message_language',
                              con=conn,
                              index_col='id',
                              dtype={'is_english': 'boolean'})
finally:
    pass

df_language.shape

(16200, 2)

In [6]:
df_language.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16200 entries, 2 to 19107
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   is_english   16200 non-null  boolean
 1   translation  16200 non-null  object 
dtypes: boolean(1), object(1)
memory usage: 284.8+ KB


In [7]:
df_language[df_language['is_english'] == False][:5]

,is_english,translation
id,,
146,False,"In the Saint Etienne area, the road to Jacmel is blocked, it is very difficult to get to Jacmel"
176,False,"In what field would you need me to speak Creole, French, half English?"
342,False,"I need some food. Thanks for your understanding. (Translator's note: are we getting the full messages? Most of them appear to be bits of sentences, like this one)"
383,False,"I need help in Creole. Contact me quickly, quickly via SMS or call, it's urgent. It is very important because today I'm going to do what I have to do over the barriers, I'm already waiting."
399,False,Don't know what to do about the staff routing part at the end


<a class="anchor" id=step3></a> 
<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">3. Set API key and connect to OpenAI</span>

To run this notebook, create an OpenAI account, load some funds only around £5 will be enough to experiment and create an API key [here]( https://platform.openai.com/account/api-keys)

Store API key under environment parameter with key 'OPENAI_API_KEY' 

In [8]:
# Setting the API key to use OPEN AI models
openai_api_key = os.environ.get('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)

<a class="anchor" id=step4></a> 
<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">4. Build the OpenAI API request in realtime</span>

Detect if messages are in English, and if not, translate them to English

In [9]:
# Identify messages that was not the checked for language yet
df_remaining = df.merge(df_language, on='id', how='left', indicator=True)
df_remaining = df_remaining[df_remaining['_merge'] == "left_only"]
df_remaining.head()

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,is_english,translation,_merge
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
19108,But it is perilously dependent on the river's hydrological cycles -- cycles that face disruptions if dams are built.,news,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,<NA>,NaN,left_only
19109,"The training, conducted by Miami-Dade Fire and Rescue Department, reached more than 100 representatives from non-governmental organizations, Mozambican military, and local fire departments and focused on SAR and relief commodity distribution capacities.",news,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,<NA>,NaN,left_only
19110,The lowlying areas behind the embankments were not flooded.,news,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,<NA>,NaN,left_only
19111,Swedish Red Cross facilitators are also providing assistance to affected families in Sweden.,news,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,<NA>,NaN,left_only
19112,"Thus far, extreme cold weather conditions have been affecting Thailand since November this year and would likely to continue until mid-February of 2009.",news,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,<NA>,NaN,left_only


In [10]:
df_remaining.shape

(9979, 40)

### Build OPENAI API request
Testing OPENAI response to see if it's working for a few messages first

In [11]:
system_prompt = """ 
  You will be provided with text about disaster responses.
  Step 1: Detect of the text is in English. Return options 'True' if the sentence is in English or 'False' if the sentence is not in English as isEnglish boolean variable
  Step 2: If sentence is not in English, translate it to English and return as text in json format
  
  Example: 'I need food' // isEnglish: True
  Example2: 'Vandaag is het zonnig'' // isEnglish: False, Translation: 'Today it is Sunny'
  """

In [12]:
# testing OPENAI response to see if it's working for a few messages first
# detect if all texts are in English, and if not translate them
mydict = {}

for idx, text in df_remaining['message'][:2].items():

  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": system_prompt,
        "temperature":0.1,
        "response_format": {
              "type": "json_object"
          },   
      },
      {
        "role": "user",
        # "content": text,
        "content": 'Vandaag is het zonnig'   
        # "content": 'I love Wilson and Maya' ,
      }
    ],
    temperature=0.1,
    top_p=1
  )

  mydict[idx] = response.choices[0].message.content
  
print(mydict)

{19108: '```json\n{\n  "isEnglish": False,\n  "Translation": "Today it is sunny"\n}\n```', 19109: '```json\n{\n  "isEnglish": False,\n  "Translation": "Today it is sunny"\n}\n```'}


<a class="anchor" id=step5></a> 
<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">5. Create OpenAI API requests in **BATCHES**</span>

Once we are happy with API response, let's kick off translation in BATCHES

Create multiple batch jobs on OpenAI platform, which will complete within 24hours. 
Due to limitations on OpenAI, only 90000 tokens can be processed in a single batch, roughly around 400 texts messages.

Running API requests are significantly cheaper but takes longer

**Important:**
The message id will be become the index, and is the main identifier of the translation

In [13]:
# Creating an array of json tasks for the next 2000 texts messages
start = 0
end = 1000
interval = 400
next = start + interval

while next <= end:
    
    # create an array of json tasks for each batch job
    tasks = []
    for index, text in df_remaining['message'][start:next].items():
    
        task = {
            "custom_id": f"task-{index}",
            "method":"POST",
            "url":"/v1/chat/completions",
            "body": {
              # This is what you would have in your Chat Completions API call
              "model":"gpt-4-turbo",
              "temperature":0.1,
              "response_format": {
                  "type": "json_object"
              },
              "messages": [
                {
                  "role": "system",
                  "content": system_prompt,
                },
                {
                  "role": "user",
                  "content": text,
                },
              ]
            }
          }
        
        tasks.append(task)
        
    # create json file and save it locally
    with open(config.path_translation_json_batchjob, 'w') as file:
        for obj in tasks:
            file.write(json.dumps(obj) + '\n') 
            
    # Uploading json file to openai platform
    batch_file = client.files.create(
        file=open(config.path_translation_json_batchjob, 'rb'),
        purpose='batch'
    )    
    
    # Creating the batch job on openai
    batch_job = client.batches.create(
        input_file_id=batch_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h"
    )    
    
    print('Batch submitted {} for records {}-{}'.format(batch_job.id, start, next))
    logger.info('Batch submitted {} for records {}-{}'.format(batch_job.id, start, next))
    
    # Check status of batch job running on openai platform
    print('Waiting for batch to start, go to sleep 5 minutes')
    time.sleep(300)
    batch_job = client.batches.retrieve(batch_job.id)
    print('Batch {}" status {}'.format(batch_job.id, batch_job.status))
    logger.info('Batch {}" status {}'.format(batch_job.id, batch_job.status))
    
    # wait for batch to complete before starting the next batch job. (CHATGPT does not allow multiple batches to run in parallel)
    while batch_job.status in ['in_progress', 'validating', 'finalizing']:
        print('Batch {} still running - going to sleep for 5 minutes'.format(batch_job.id))
        logger.info('Batch {} still running - going to sleep for 5 minutes'.format(batch_job.id))
        time.sleep(300)
        batch_job = client.batches.retrieve(batch_job.id)
    
    # when batch is completed, set counters to kick off the next batch job of 400 requests   
    if not batch_job.status == 'failed':
        start = start + interval
        next = next + interval

Batch submitted batch_LlpEd0utraYayrtmuvfCOqvZ for records 0-400
Waiting for batch to start, go to sleep 5 minutes
Batch batch_LlpEd0utraYayrtmuvfCOqvZ" status in_progress
Batch batch_LlpEd0utraYayrtmuvfCOqvZ still running - going to sleep for 5 minutes
Batch submitted batch_p2DLuS5uKh6BQxBEBskdDQEb for records 400-800
Waiting for batch to start, go to sleep 5 minutes
Batch batch_p2DLuS5uKh6BQxBEBskdDQEb" status completed


<a class="anchor" id=step6></a> 
<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">6. Load and Analyze API batch results into JSON file</span>

Download all batch results from OpenAI and accumulate all API results in a local JSON file

In [14]:
# get all batch jobs that were submitted from openai
# batch_jobs = []
batch_jobs = client.batches.list(limit=100)
print('Number of batch jobs retrieved: {}'.format(len(batch_jobs.data)))

Number of batch jobs retrieved: 72


In [15]:
# select batches to process
batches = []
for batch in batch_jobs.data:
   if ((batch.status == 'completed') & 
        (batch.request_counts.failed == 0) & 
        (batch.request_counts.total > 10)):
            batches.append(batch.id)
            print(batch.id, batch.status, batch.request_counts.completed, batch.request_counts.failed) 

batch_p2DLuS5uKh6BQxBEBskdDQEb completed 400 0
batch_LlpEd0utraYayrtmuvfCOqvZ completed 400 0
batch_HFjgaBPobSG3oxVZQvkcQDJo completed 400 0
batch_8Vg8BEsadY4nmaZppllkffrF completed 400 0
batch_9KmEZ7TTk0dUIORNsdQoI26d completed 400 0
batch_lruWCxoOZFaPODmHrK49GNvV completed 400 0
batch_uYn2OAjWahdndiMmdgRb8Rlz completed 400 0
batch_flqNjEuIjFKEd9SUwmPzwlTo completed 400 0
batch_eoRTRXDt0mheHpbWjeW8uQKE completed 400 0
batch_ov3GATjBCu4T1IOk75zqN5AK completed 400 0
batch_00nrAmQDdiwLujbhUDLh3Z1X completed 400 0
batch_MXtwNlcS4bga5pp3BKXErCyx completed 400 0
batch_0iP3N0CeNDnW4FmJ3mXTRLUc completed 400 0
batch_zhOCvJWmcxs8WEbtc6zxXTli completed 400 0
batch_4SBS9UWZAhESFSouylcAtfxt completed 400 0
batch_OtRp27uyP9MESWJvACeHYRVa completed 400 0
batch_Umpx2i1ksQbHNavqFBtQmGCw completed 400 0
batch_m6YkBnqtIYz7Mh6McZhnuJpJ completed 400 0
batch_eVRWQrUVnEaD28KPb9btFTkD completed 400 0
batch_alRKHlkCdds2N7GzfJSuoz1u completed 400 0
batch_pqs3fOnfysjliQbhDQ2T6o9S completed 400 0
batch_ITarlL3

In [16]:
# Download batch content from OPENAI and consolidate all api results locally into a json file
# OPENAI will delete files from batches after 30 days, so we might not be able to retrieve old content
# file batch_job_resuts will still contain all those old details so don't delete it, just append

# first clear the file if it exists
# open(config.path_translation_json_batchjob_result, 'w').close()

# append contents of all batches to local json results file
for batch in batches:
    batch_job = client.batches.retrieve(batch) 
    try:        
        result = client.files.content(batch_job.output_file_id).content
        with open(config.path_translation_json_batchjob_result, 'ab') as file:
            file.write(result)   
    except:
        # print('Batch file {} output already deleted on openai'.format(batch))
        logger.info('Batch file {} output already deleted on openai'.format(batch))
        

In [17]:
# Loading all json api data from locally saved json file
results = []
with open(config.path_translation_json_batchjob_result, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.replace('\n', '').strip())
        results.append(json_object)

In [18]:
# Reading only the first results as a test if API worked
for res in results[100:106]:
    task_id = res['custom_id']
    # Getting index from task id
    index = task_id.split('-')[-1]
    result = res['response']['body']['choices'][0]['message']['content']
    movie = df.loc[int(index)]
    description = movie['message']
    # title = movie['Series_Title']
    print(f"\nMESSAGE: {index}-{description}\n\nRESULT: {result}")
    print("\n----------------------------\n")


MESSAGE: 8084-Darling, I love you even if you once had forgot, you make me suffer I do not relax so that you understand me. Thank you 

RESULT: {
  "isEnglish": true
}

----------------------------


MESSAGE: 8085-what must we do when don't feel good we'd like to get about of our mind 

RESULT: {
  "isEnglish": true
}

----------------------------


MESSAGE: 8086-Need help in Corail department Grand'Anse. 

RESULT: {
  "isEnglish": true
}

----------------------------


MESSAGE: 8087-Can you give me a stock market of studies in the communication domain? While hoping your response, receive my greetings the better ones 

RESULT: {
  "isEnglish": true
}

----------------------------


MESSAGE: 8088-Information requiere about of the earthquake. 

RESULT: {
  "isEnglish": true
}

----------------------------


MESSAGE: 8089-I'm hungry,I don't have food to eat,I don't have home,clothes, I count on your supports and help,thank you so much. 

RESULT: {
  "isEnglish": true
}

-----------------

<a class="anchor" id=step7></a> 
<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">7. Load API results from JSON into pandas dataframe</span>

The API return 2 responses we are interested in:
1) A Boolean variable `isEnglish` to indicate if the message are in English or not
2) A text variable `translation` in English if the message was in another language

Save both these responses in a separate dictionary and create a dataframe with 2 columns:
- is_english
- translation

In [19]:
# Load all responses into a dictionaries
isEnglishs = {}
translatedTexts = {}

for res in results:
    task_id = res['custom_id']
    # Get unique message id from task id
    index = int(task_id.split('-')[-1])
    # get response content and strip of new line indicators
    result = res['response']['body']['choices'][0]['message']['content']
    result = result.replace('\n', '').strip()
    result = result.replace('\t', '').strip()
    # get original message
    df_tmp = df.loc[index]
    description = df_tmp['message']
    translation = ''
    isEnglish = ''

    try:
        dict_object = json.loads(result)
        # isEnglish = bool(dict_object['isEnglish'])
        isEnglish = dict_object['isEnglish']
    except:
        pass

    try:
        translation = dict_object['Translation']       
    except:
        pass
    
    isEnglishs[index] = isEnglish
    translatedTexts[index] = translation
    

In [20]:
results[0]

{'id': 'batch_req_8kb0xLtfRWdxvI6NZtYdKfIn',
 'custom_id': 'task-7976',
 'response': {'status_code': 200,
  'request_id': 'a8d81f4f0ed5cfb41a472d88bfa41e81',
  'body': {'id': 'chatcmpl-9M8EfKZZxwQD140JEZKgHrvjPZwBL',
   'object': 'chat.completion',
   'created': 1715061901,
   'model': 'gpt-4-turbo-2024-04-09',
   'choices': [{'index': 0,
     'message': {'role': 'assistant', 'content': '{\n  "isEnglish": true\n}'},
     'logprobs': None,
     'finish_reason': 'stop'}],
   'usage': {'prompt_tokens': 150,
    'completion_tokens': 9,
    'total_tokens': 159},
   'system_fingerprint': 'fp_ea6eb70039'}},
 'error': None}

<a class="anchor" id=step8></a> 
<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">8. Clean translations and save to SQLite database</span>

In [21]:
# create dataframe
data = {'is_english': isEnglishs,
        'translation': translatedTexts}

df_translation = pd.DataFrame.from_dict(data,
                                         orient='columns',
                                         )
df_translation.index.name = 'id'
df_translation.head()

,is_english,translation
id,,
7976,True,
7977,True,
7978,True,
7979,True,
7980,True,


In [22]:
df_translation.shape

(17000, 2)

In [23]:
# Add message for analysis
df_translated_enhanced = df_translation.merge(df[['message']], on='id', how='inner')
df_translated_enhanced.head()

,is_english,translation,message
id,,,
7976,True,,"MAY GOD BLESS HAITI,CHILY AND CHINA. THANK'S"
7977,True,,"We in Canada turjo quote, we need food, water and tents. count on your participation"
7978,True,,Thank you for all the information you gave me.
7979,True,,"We geet,the organisation for the good work for all haitian people:We that living in la sous tijo,for the city of canada we need some aids i think you could."
7980,True,,FLOOD AT CAYES. HELP US EMERGENCY


In [24]:
df_translated_enhanced.shape

(17000, 3)

In [25]:
df_translated_enhanced.is_english.value_counts()

is_english
True     16655
False      343
True         1
False        1
Name: count, dtype: int64

In [26]:
df_translated_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 7976 to 19554
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   is_english   17000 non-null  object
 1   translation  17000 non-null  object
 2   message      17000 non-null  object
dtypes: object(3)
memory usage: 531.2+ KB


In [27]:
# remove spaces
df_translated_enhanced['is_english'] = df_translated_enhanced['is_english'].map(lambda x: str(x).replace(' ', '').capitalize())
df_translated_enhanced.is_english.value_counts()

is_english
True     16656
False      344
Name: count, dtype: int64

In [28]:
df_translated_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 7976 to 19554
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   is_english   17000 non-null  object
 1   translation  17000 non-null  object
 2   message      17000 non-null  object
dtypes: object(3)
memory usage: 531.2+ KB


In [29]:
# convert to boolean
df_translated_enhanced['is_english'] = df_translated_enhanced['is_english'].map(lambda x: True if str(x) == 'True' else False)
    
# df_translated_enhanced['is_english'] = np.where(df_translated_enhanced['is_english'].str.contains('True'), True, False)

In [30]:
df_translated_enhanced.is_english.value_counts()

is_english
True     16656
False      344
Name: count, dtype: int64

In [31]:
df_translated_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 7976 to 19554
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   is_english   17000 non-null  bool  
 1   translation  17000 non-null  object
 2   message      17000 non-null  object
dtypes: bool(1), object(2)
memory usage: 415.0+ KB


In [32]:
# how many texts were not in English ?
print('Messages not in English:', len(df_translated_enhanced[df_translated_enhanced.is_english == False]))
df_translated_enhanced[df_translated_enhanced.is_english == False].sample(10)

Messages not in English: 344


,is_english,translation,message
id,,,
7463,False,The person doesn't make space among the letters.,The person doesn't make space among the letters.
14266,False,"Eid Mubarak to you and your entire team. Sir, we have a request that you visit our city once or send your representative so that you will know about all our conditions! Sir, thousands of people did not receive the Watan card.","Eid Mubarak to you and your entire team. Sir, we have a request that you visit our city once or send your representative so that you will know about all our conditions! Sir, thousands of people did not receive the Watan card."
8497,False,The text provided does not contain any recognizable language content.,The text provided does not contain any recognizable language content.
13501,False,I just read a paper on Interseismic strain accumulation measured by GPS in the seismic gap between Constitución and Concepción in Chile. Uf.,I just read a paper on Interseismic strain accumulation measured by GPS in the seismic gap between Constitución and Concepción in Chile. Uf.
14155,False,address mushroom. d.s.s. Rajanpur?,address mushroom. d.s.s. Rajanpur?
13949,False,"Our non-governmental organizations, what work do they do and what thing belongs to which area? Please, government (SRSO) organization (Crecent Hillah) Red (Creation Foundation) (Pirbhat)","Our non-governmental organizations, what work do they do and what thing belongs to which area? Please, government (SRSO) organization (Crecent Hillah) Red (Creation Foundation) (Pirbhat)"
14564,False,"Sir, I request that we do justice to the United Nations, have a right to the national highway, and if we are really affected by the flood, help the affected people.","Sir, I request that we do justice to the United Nations, have a right to the national highway, and if we are really affected by the flood, help the affected people."
7397,False,"Do something for us NGO, answer","Do something for us NGO, answer"
14156,False,Is there any flood in Rajanpur?,Is there any flood in Rajanpur?


In [33]:
funny_texts = [
    'The text provided does not contain any recognizable language or meaningful content.',
    'This SMS is from another language.',
    'It is not necessary to translate this message because this message is sentimental.',
    'NOTES: I cannot translate this message because it is not complete.',
    'NOTES: this person wants to translate two words',
    'I already translated this one.',
    'NOTES: I already translated the message is not important to translate.'
]

df_translated_enhanced[df_translated_enhanced['translation'].isin(funny_texts)]

,is_english,translation,message
id,,,
3983,False,I already translated this one.,I already translated this one.
11100,False,NOTES: this person wants to translate two words,NOTES: this person wants to translate two words
9986,False,It is not necessary to translate this message because this message is sentimental.,It is not necessary to translate this message because this message is sentimental.
9287,False,The text provided does not contain any recognizable language or meaningful content.,The text provided does not contain any recognizable language or meaningful content.
9348,False,NOTES: I cannot translate this message because it is not complete.,NOTES: I cannot translate this message because it is not complete.
9066,False,This SMS is from another language.,This SMS is from another language.
8192,False,The text provided does not contain any recognizable language or meaningful content.,The text provided does not contain any recognizable language or meaningful content.


In [34]:
# if text is not marked as English ,but no translation was given, set the language back to English
error_index  = df_translated_enhanced[(df_translated_enhanced['is_english'] == False) & 
                                      (df_translated_enhanced['translation'].map(lambda x: len(x) == 0))].index
len(error_index)

4

In [35]:
df_translated_enhanced.loc[error_index]

,is_english,translation,message
id,,,
1244,False,,
14428,False,,
14315,False,,
11161,False,,


In [36]:
# do we have duplicates ?
df_translated_enhanced.duplicated().sum()

8

In [37]:
# drop duplicated translations if it exists
df_translated_enhanced =  df_translated_enhanced[~df_translated_enhanced.index.duplicated(keep='first')]
df_translated_enhanced.shape

(17000, 3)

In [38]:
df_translated_enhanced = df_translated_enhanced.sort_index()
df_translated_enhanced.head()

,is_english,translation,message
id,,,
2,True,,Weather update - a cold front from Cuba that could pass over Haiti
7,True,,Is the Hurricane over or is it not over
8,True,,Looking for someone but no name
9,True,,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
12,True,,"says: west side of Haiti, rest of the country today and tonight"


### Save the dataset to sqlite database.

In [39]:
df_translated_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 2 to 20002
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   is_english   17000 non-null  bool  
 1   translation  17000 non-null  object
 2   message      17000 non-null  object
dtypes: bool(1), object(2)
memory usage: 415.0+ KB


In [40]:
# make sure is_english contain only values True and False
print(df_translated_enhanced.is_english.unique())

[ True False]


In [41]:
# save to csv file

df_translation.to_csv(config.path_log_translation)

In [42]:
# Drop tmp column before we save it
df_translated_enhanced.drop(['message'], inplace=True, axis=1)
df_translated_enhanced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 2 to 20002
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   is_english   17000 non-null  bool  
 1   translation  17000 non-null  object
dtypes: bool(1), object(1)
memory usage: 282.2+ KB


In [43]:
# add to existing sqlite database
df_translated_enhanced.to_sql('message_language', engine, index=True, if_exists='replace')

17000

<a class="anchor" id=step8></a> 
<span style="color:blue; font-size:25px; line-height:30px; font-family: Calibri;">9. Update `messages` dataset with translations and update SQLite database</span>

In [44]:
# load cleaned message data from database
engine = create_engine(config.path_database)
conn = engine.connect()
df = pd.read_sql('select * from messages', con=conn, index_col='id')
df.head()

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that could pass over Haiti,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country today and tonight",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26179 entries, 2 to 30265
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   message                 26179 non-null  object
 1   genre                   26179 non-null  object
 2   related                 26179 non-null  int64 
 3   request                 26179 non-null  int64 
 4   offer                   26179 non-null  int64 
 5   aid_related             26179 non-null  int64 
 6   medical_help            26179 non-null  int64 
 7   medical_products        26179 non-null  int64 
 8   search_and_rescue       26179 non-null  int64 
 9   security                26179 non-null  int64 
 10  military                26179 non-null  int64 
 11  water                   26179 non-null  int64 
 12  food                    26179 non-null  int64 
 13  shelter                 26179 non-null  int64 
 14  clothing                26179 non-null  int64 
 15  money  

In [46]:
# read message language detection and translation to English, if it exists
df_language = pd.DataFrame()
try:
    df_language = pd.read_sql('select * from message_language',
                              con=conn,
                              index_col='id',
                              dtype={'is_english': 'boolean'}
                              )
finally:
    pass

df_language.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 2 to 20002
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   is_english   17000 non-null  boolean
 1   translation  17000 non-null  object 
dtypes: boolean(1), object(1)
memory usage: 298.8+ KB


In [47]:
df_language.sort_index().head()

,is_english,translation
id,,
2,True,
7,True,
8,True,
9,True,
12,True,


In [48]:
# merge dataframes
if len(df_language) > 0:
    df = df.merge(df_language, on='id', how='left' )
    
df.head()

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,is_english,translation
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that could pass over Haiti,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,
7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,True,
8,Looking for someone but no name,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,
9,UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.,direct,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,True,
12,"says: west side of Haiti, rest of the country today and tonight",direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,


In [49]:
df['original_message'] = df['message']

In [50]:
df.is_english.value_counts(dropna=False)

is_english
True     16656
<NA>      9179
False      344
Name: count, dtype: Int64

In [51]:
# For messages with no translation or language detection, set is_english = True
df['is_english'] = np.where(df['is_english'].isna(), True, df['is_english'])
df.is_english.value_counts(dropna=False)

is_english
True     25835
False      344
Name: count, dtype: int64

In [52]:
# display some translations
df[df['is_english'] == False][['original_message', 'is_english', 'translation']].head(5)

,original_message,is_english,translation
id,,,
146,"In the Saint Etienne area, the road to Jacmel is blocked, it is very difficult to get to Jacmel",False,"In the Saint Etienne area, the road to Jacmel is blocked, it is very difficult to get to Jacmel"
176,"In what field would you need me to speak Creole, French, half English?",False,"In what field would you need me to speak Creole, French, half English?"
342,"I need some food. Thanks for your understanding. (Translator's note: are we getting the full messages? Most of them appear to be bits of sentences, like this one)",False,"I need some food. Thanks for your understanding. (Translator's note: are we getting the full messages? Most of them appear to be bits of sentences, like this one)"
383,"I need help in Creole. Contact me quickly, quickly via SMS or call, it's urgent. It is very important because today I'm going to do what I have to do over the barriers, I'm already waiting.",False,"I need help in Creole. Contact me quickly, quickly via SMS or call, it's urgent. It is very important because today I'm going to do what I have to do over the barriers, I'm already waiting."
399,Don't know what to do about the staff routing part at the end,False,Don't know what to do about the staff routing part at the end


In [53]:
# replace message with translation, if message is flagged as not being in English
df['message'] =  np.where((df['is_english'] == False) & (~df['translation'].isnull()),
                           df['translation'],
                           df['original_message'])

In [54]:
# make sure messages already in English are untouched
cols = ['original_message', 'is_english', 'translation', 'message']
df[df['is_english'] == True][cols].sample(5)

,original_message,is_english,translation,message
id,,,,
946,"its that i have a lot of problems. my house has colapsed, i dont have clothes, I p. ..",True,,"its that i have a lot of problems. my house has colapsed, i dont have clothes, I p. .."
11103,"i can not stand no more, u got every card that i put on my phone with some messages .please, give me a chance cause i can not get no more on my phone.",True,,"i can not stand no more, u got every card that i put on my phone with some messages .please, give me a chance cause i can not get no more on my phone."
18077,Survivors say they were blinded for half-an-hour by the dust from landfalls.,True,,Survivors say they were blinded for half-an-hour by the dust from landfalls.
24226,"They will pack up whatever belongings they managed to save after the 10-metre (33-foot) waves swept through their homes and move to wooden barracks in 37 different areas across Aceh, a process that will take two weeks or so, the official said.",True,NaN,"They will pack up whatever belongings they managed to save after the 10-metre (33-foot) waves swept through their homes and move to wooden barracks in 37 different areas across Aceh, a process that will take two weeks or so, the official said."
26744,"Providing sufficient shelter, winter-proof tents and heaters,"" said Markku Niskala, Secretary-General of the International Federation of Red Cross and Red Crescent Societies (IFRC), visiting Balakot.",True,NaN,"Providing sufficient shelter, winter-proof tents and heaters,"" said Markku Niskala, Secretary-General of the International Federation of Red Cross and Red Crescent Societies (IFRC), visiting Balakot."


In [55]:
# check that messages not in English were replaced
df[df['is_english'] == False][cols].sample(10)

,original_message,is_english,translation,message
id,,,,
1569,"In the name of the youth in action from Croix des Missions, we are requesting...",False,"In the name of the youth in action from Croix des Missions, we are requesting...","In the name of the youth in action from Croix des Missions, we are requesting..."
508,We are desperate for food and water. Our organization is Kole Zepol to deliver,False,We are desperate for food and water. Our organization is Kole Zepol to deliver,We are desperate for food and water. Our organization is Kole Zepol to deliver
14065,(Peace be upon you) Union Council Rojhan East Rojhan Saheja destroyed People Please help us,False,(Peace be upon you) Union Council Rojhan East Rojhan Saheja destroyed People Please help us,(Peace be upon you) Union Council Rojhan East Rojhan Saheja destroyed People Please help us
10530,Djee Boby is a man who has implemented Rap Creole in Haiti.,False,Djee Boby is a man who has implemented Rap Creole in Haiti.,Djee Boby is a man who has implemented Rap Creole in Haiti.
14219,The sky is blue,False,The sky is blue,The sky is blue
14598,"TAUSEEF KHAN S/O AKBAR DIN MIANWALI TEH ESSA KHEL VIA KALA BAGH P/O TOLA MANGLI VILLAGE MUHABBAT KHEL sir I have been affected by the flood, but what can I do",False,"TAUSEEF KHAN S/O AKBAR DIN MIANWALI TEH ESSA KHEL VIA KALA BAGH P/O TOLA MANGLI VILLAGE MUHABBAT KHEL sir I have been affected by the flood, but what can I do","TAUSEEF KHAN S/O AKBAR DIN MIANWALI TEH ESSA KHEL VIA KALA BAGH P/O TOLA MANGLI VILLAGE MUHABBAT KHEL sir I have been affected by the flood, but what can I do"
11884,Earthquake in Haiti http bit.ly 8DoZ21,False,Earthquake in Haiti http bit.ly 8DoZ21,Earthquake in Haiti http bit.ly 8DoZ21
9162,cyclone,False,cyclone,cyclone
11100,NOTES: this person wants to translate two words,False,NOTES: this person wants to translate two words,NOTES: this person wants to translate two words


In [56]:
# finally lets drop the columns we no longer need
df = df.drop(['original_message', 'is_english', 'translation'], axis=1)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26179 entries, 2 to 30265
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   message                 26179 non-null  object
 1   genre                   26179 non-null  object
 2   related                 26179 non-null  int64 
 3   request                 26179 non-null  int64 
 4   offer                   26179 non-null  int64 
 5   aid_related             26179 non-null  int64 
 6   medical_help            26179 non-null  int64 
 7   medical_products        26179 non-null  int64 
 8   search_and_rescue       26179 non-null  int64 
 9   security                26179 non-null  int64 
 10  military                26179 non-null  int64 
 11  water                   26179 non-null  int64 
 12  food                    26179 non-null  int64 
 13  shelter                 26179 non-null  int64 
 14  clothing                26179 non-null  int64 
 15  money  

In [58]:
df.to_sql('messages', engine, index=True, if_exists='replace')

26179